## Load data

In [1022]:
import pandas as pd 
import numpy as np  
import seaborn as sns   
import matplotlib.pyplot as plt 

In [1023]:
sales_df = pd.read_csv("/Users/Marcy_Student/Desktop/Marcy_Projects/Mod1_Final_Project/data/raw/cafe_sales.csv", date_parser=["Transaction Date"],)

/var/folders/h0/75t8pd8571n2l1zzjcyqp7kh0000gp/T/ipykernel_79081/664495163.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  sales_df = pd.read_csv("/Users/Marcy_Student/Desktop/Marcy_Projects/Mod1_Final_Project/data/raw/cafe_sales.csv", date_parser=["Transaction Date"],)


## Getting a Quick info of the dataset

In [1024]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [1025]:
sales_df.head(15)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


# Data Cleaning and Preparation 🧹
1. convertion of the `Item`, `Quantity`, `Price Per Unit` columns to numeric and `Transaction Date` to Date time.

In [1026]:
sales_df["Quantity"] = pd.to_numeric(sales_df["Quantity"], errors="coerce")
sales_df["Price Per Unit"] = pd.to_numeric(sales_df["Price Per Unit"], errors="coerce")
sales_df["Total Spent"] = pd.to_numeric(sales_df["Total Spent"], errors="coerce")
sales_df["Transaction Date"] = pd.to_datetime(sales_df["Transaction Date"], errors="coerce")
#sales_df.info()

2. Checking for missing values inside the data frame. 

In [1027]:
sales_df.isna().sum()
#we have a lot of data missing, let's try to clean it and replace it 🧹 

Transaction ID         0
Item                 333
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      2579
Location            3265
Transaction Date     460
dtype: int64

In [1028]:
#sales_df['Price Per Unit'].unique()
sales_df_columns = ['Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date']
for col in sales_df_columns:
    unique_values = sales_df[col].unique()
    print(f"\nThe unique values inside {col} are: {unique_values}")
    print(sales_df[col].value_counts())


The unique values inside Item are: ['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'UNKNOWN' 'Sandwich' nan
 'ERROR' 'Juice' 'Tea']
Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64

The unique values inside Quantity are: [ 2.  4.  5.  3.  1. nan]
Quantity
5.0    2013
2.0    1974
4.0    1863
3.0    1849
1.0    1822
Name: count, dtype: int64

The unique values inside Price Per Unit are: [2.  3.  1.  5.  4.  1.5 nan]
Price Per Unit
3.0    2429
4.0    2331
2.0    1227
5.0    1204
1.0    1143
1.5    1133
Name: count, dtype: int64

The unique values inside Total Spent are: [ 4.  12.   nan 10.  20.   9.  16.  15.  25.   8.   5.   3.   6.   2.
  1.   7.5  4.5  1.5]
Total Spent
6.0     979
12.0    939
3.0     930
4.0     923
20.0    746
15.0    734
8.0     677
10.0    524
2.0     497
9.0     479
5.0     468
16.0    444
25.0    259
7.5     237
1.0 

## Handling Missing Values
Let's fill the missing item values using the `Price Per Unit` column. We will be using the price of the item to identify which item should be in the empty spot.

However, because the prices are not all unique to each item, there are two scenarios when doing this:

1. The price is unique and only has one item for that price: We directly use a mapping function to replace the NaNs.
2. The price is not unique and has multiple items for that price: We figure out which item is more frequent and use that to fill the space.


In [1029]:
sales_df["Price Per Unit"].isna().sum()

np.int64(533)

- First, let's replace the 'ERROR' and 'UNKNOWN' values by NaN in the Item, Payement Method and Location. 


In [1030]:
sales_col = ['Item','Payment Method','Location']
for col in sales_col:
   sales_df[col] = sales_df[col].replace(['ERROR', 'UNKNOWN'], np.nan)


- Now 'ERROR', 'UNKNOWN' in `Item`, `Payement Method`, and `Location` are replaced by NaN. 


In [1031]:
sales_df["Item"].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', nan, 'Sandwich',
       'Juice', 'Tea'], dtype=object)

In [ ]:
# A dictionnary containing the price per unit for each item
price_dict = {
    'Coffee': 2.0,
    'Tea': 1.5,
    'Sandwich': 4.0,
    'Salad': 5.0,
    'Cake': 3.0,
    'Cookie': 1.0,
    'Smoothie': 4.0,
    'Juice': 3.0
}

We will now use this price_dict dictionary to fill the data that we can fill. Wherever the price is one of the keys in the price_dict dictionary, we will replace the NaN with the appropriate data.

In [1033]:
sales_df["Price Per Unit"] = sales_df["Price Per Unit"].fillna(sales_df["Item"].map(price_dict))

In [1034]:
sales_df["Price Per Unit"].isna().sum()
#Now we only have 54 missing values!

np.int64(54)

In [1035]:
price_to_name_dict = {
  2.0: 'Coffee',
    1.5: 'Tea',
    5.0: 'Salad',
    1.0: 'Cookie',    
}
sales_df['Item'] = sales_df['Item'].fillna(sales_df['Price Per Unit'].map(price_to_name_dict))


In [1036]:
sales_df.isna().sum()

Transaction ID         0
Item                 501
Quantity             479
Price Per Unit        54
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

### Finding the missing values in `Total Spent` using the `Price Per Unit` and `Quantity`

In [1037]:
#Total spent using total quantity and total price per unit 
total = sales_df['Total Spent'].isna() & sales_df['Quantity'].notna() & sales_df['Price Per Unit'].notna()
sales_df.loc[total, 'Total Spent'] = sales_df.loc[total, 'Quantity'] * sales_df.loc[total, 'Price Per Unit']

In [1038]:
#Quantity using total quantity and total price per unit 
Quantity = sales_df["Quantity"].isna() & sales_df["Total Spent"].notna() & sales_df["Price Per Unit"].notna()
sales_df.loc[Quantity, "Quantity"] = sales_df.loc[Quantity, "Total Spent"] / sales_df.loc[Quantity, "Price Per Unit"]

In [1039]:
#Price Per unit using total quantity and total price per unit 
Price_PU = sales_df["Price Per Unit"].isna() & sales_df["Total Spent"].notna() & sales_df["Quantity"].notna()
sales_df.loc[Price_PU, "Price Per Unit"] = sales_df.loc[Price_PU, "Total Spent"] / sales_df.loc[Price_PU, "Quantity"]

In [1040]:
#Let's try to map again to fill the missing items. 
sales_df['Item'] = sales_df['Item'].fillna(sales_df['Price Per Unit'].map(price_to_name_dict))


In [1041]:
sales_df.isna().sum()
#sales_df.info()

Transaction ID         0
Item                 480
Quantity              23
Price Per Unit         6
Total Spent           23
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

Now all the remaining missing values in `Item`, `Quantity`, `Price Per Unit` and `Total Payement` are less than 5% of the total values, we can drop them because it they don't going to affect our analysis much
We were not able to fill in the remaing missing values in `Item` because "Juice" and "Cake" have the same price, they both cost $3. We can't map that using the Dictionary.

In [852]:
# Find the five percent threshold
threshold = len(sales_df) * 0.05

# Create a filter
cols_to_drop = sales_df.columns[sales_df.isna().sum() <= threshold]

# Drop missing values for columns below the threshold
sales_df.dropna(subset=cols_to_drop, inplace=True)


In [853]:
sales_df.isna().sum()


Transaction ID         0
Item                   0
Quantity               0
Price Per Unit         0
Total Spent            0
Payment Method      2870
Location            3592
Transaction Date       0
dtype: int64

### Checking for duplicate values 

In [1042]:
sales_df.duplicated().sum()
#We don't have any duplicate value, that's a good news 🥳🥳🥳

np.int64(0)

### Let's create two new columns for the date

In [1043]:
sales_df["Month"] = sales_df["Transaction Date"].dt.strftime("%B") #strftime extract the year from the date. 
sales_df["Day Of The Week"] =  sales_df["Transaction Date"].dt.day_name() # dt.date_name extract the name of the day.

In [1044]:
sales_df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Month,Day Of The Week
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08,September,Friday
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16,May,Tuesday
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19,July,Wednesday
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27,April,Thursday
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11,June,Sunday


In [1046]:
sales_df.to_csv("cafe_sales_cleaned.csv",index=False)